In [ ]:
!pip install -q \
  langchain \
  langchain-community \
  langchain-text-splitters \
  chromadb \
  sentence-transformers \
  ollama \
  pydantic \
  python-dotenv \
  chromadb


In [ ]:
! /Users/sohampatil/.pyenv/versions/3.9.10/bin/python -m pip install --upgrade pip
! /Users/sohampatil/.pyenv/versions/3.9.10/bin/python -m pip install langchain-groq


In [ ]:
import os
import subprocess
from pathlib import Path
from typing import List

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.schema import Document


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip langchain-groq


In [ ]:
PROJECT_ROOT = Path("/Users/sohampatil/Documents/Rewear")

assert PROJECT_ROOT.exists(), "Project root not found"
print("Project root set to:", PROJECT_ROOT)


In [ ]:
def load_repo_documents(root: Path) -> List[Document]:
    docs = []
    for path in root.rglob("*"):
        if any(ex in path.parts for ex in EXCLUDE_DIRS):
            continue
        if path.suffix in ALLOWED_EXTENSIONS and path.is_file():
            try:
                content = path.read_text(errors="ignore")
                docs.append(
                    Document(
                        page_content=content,
                        metadata={"source": str(path.relative_to(root))}
                    )
                )
            except Exception as e:
                print("Skipped:", path, e)
    return docs

documents = load_repo_documents(PROJECT_ROOT)
print("Loaded documents:", len(documents))


In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)
chunks=splitter.split_documents(documents)
print("Created chunks:", len(chunks))

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade torch sentence-transformers


In [ ]:
!pip install langchain-huggingface sentence-transformers

In [ ]:
import os
import requests
from langchain.schema import Document

HF_TOKEN = os.environ["HF_TOKEN"]

class HFEmbeddings:
    def __init__(self, model_name, api_key):
        self.model_name = model_name
        self.api_key = api_key
        self.api_url = f"https://api-inference.huggingface.co/models/{model_name}"

    def embed_documents(self, texts):
        headers = {"Authorization": f"Bearer {self.api_key}"}
        embeddings = []
        for text in texts:  # Send one at a time to avoid JSON errors
            response = requests.post(
                self.api_url,
                headers=headers,
                json={"inputs": text, "options": {"wait_for_model": True}}
            )
            if response.status_code != 200:
                raise ValueError(f"HF API error {response.status_code}: {response.text}")
            result = response.json()
            if isinstance(result, list):
                embeddings.append(result[0])
            else:
                raise ValueError(f"Unexpected response format: {result}")
        return embeddings

    def embed_query(self, text):
        return self.embed_documents([text])[0]



In [ ]:
import requests
import os
import time

class HFEmbeddings:
    def __init__(self, model_name):
        self.model_name = model_name
        self.api_key = os.environ["HF_TOKEN"]
        self.api_url = f"https://router.huggingface.co/hf-inference/models/{model_name}"

    def _call(self, inputs):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        response = requests.post(
            self.api_url,
            headers=headers,
            json={"inputs": inputs}
        )

        if response.status_code == 503:
            time.sleep(3)
            return self._call(inputs)

        if response.status_code != 200:
            raise ValueError(f"HF error {response.status_code}: {response.text}")

        return response.json()

    def embed_documents(self, texts):
        result = self._call(texts)

        # Router returns: List[List[float]]
        if not isinstance(result, list) or not isinstance(result[0], list):
            raise ValueError(f"Unexpected embedding format: {result}")

        return result

    def embed_query(self, text):
        return self.embed_documents([text])[0]


In [ ]:
from langchain.schema import Document

texts = [
    "CI/CD pipelines explanation",
    "Kubernetes deployment strategies",
    "Monitoring and alerting systems"
]

docs = [Document(page_content=t) for t in texts]

embeddings = HFEmbeddings("sentence-transformers/all-MiniLM-L6-v2")

for i, d in enumerate(docs):
    emb = embeddings.embed_query(d.page_content)
    print(f"Chunk {i+1} embedding size:", len(emb))


In [ ]:
!pip install -U sentence-transformers chromadb


In [ ]:
import os
from langchain.schema import Document

ALLOWED_EXTENSIONS = {
    ".tf", ".yaml", ".yml", ".md", ".sh", ".json", ".py", ".js"
}

EXCLUDE_DIRS = {
    ".terraform", "node_modules", ".git", "build", "dist"
}

project_dirs = [
    "k8s", "modules", "rewear-backend", "rewear-frontend", "terraform-backend"
]

docs = []

for d in project_dirs:
    for root, dirs, files in os.walk(d):
        dirs[:] = [x for x in dirs if x not in EXCLUDE_DIRS]

        for f in files:
            if os.path.splitext(f)[1] in ALLOWED_EXTENSIONS:
                path = os.path.join(root, f)
                try:
                    with open(path, "r", encoding="utf-8") as file:
                        text = file.read()
                        if text.strip():
                            docs.append(
                                Document(
                                    page_content=text,
                                    metadata={"source": path}
                                )
                            )
                except:
                    pass

print(f"✅ Clean documents loaded: {len(docs)}")


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1️⃣ Initialize embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 2️⃣ Split large documents into smaller chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,   # 1k chars per chunk
    chunk_overlap=200  # 100 chars overlap for context
)

chunked_docs = []
for doc in docs:
    chunks = splitter.split_text(doc.page_content)
    chunked_docs.extend([
    Document(page_content=c, metadata=doc.metadata)
    for c in chunks
])

print(f"✅ Total chunks to embed: {len(chunked_docs)}")

# 3️⃣ Create Chroma vector store
vectordb = Chroma(
    collection_name="rewear-devops",
    embedding_function=embeddings,
    persist_directory="./vectorstore"
)

# 4️⃣ Add documents in **small batches**
batch_size = 20
for i in range(0, len(chunked_docs), batch_size):
    batch = chunked_docs[i:i+batch_size]
    vectordb.add_documents(batch)
    print(f"✅ Added batch {i//batch_size + 1} / {(len(chunked_docs)+batch_size-1)//batch_size}")

print("✅ Vector store created and ready for similarity search")


In [ ]:
from langchain_groq import ChatGroq


In [ ]:
!pip install python-dotenv


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Access the API key
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Example usage
print("GROQ_API_KEY loaded:", bool(GROQ_API_KEY))


In [ ]:
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
print(" LLM loaded successfully")

In [ ]:
def rag_query(question: str, k: int = 6) -> str:
    docs = vectordb.similarity_search(question, k=k)

    context = "\n\n".join(
        f"[{d.metadata.get('source', 'unknown')}]\n{d.page_content}"
        for d in docs
    )

    prompt = f"""
You are a senior DevOps engineer.

Use ONLY the repo context below.
Be precise. No hallucination.

Context:
{context}

Question:
{question}

Answer:
"""
    return llm.invoke(prompt)


In [ ]:
def terraform_agent(question: str):
    return rag_query(
        f"""
You are a Terraform infrastructure expert.

Rules:
- Answer ONLY if the question involves Terraform, infrastructure, or IaC
- Focus on VPC, EKS, IAM, S3, CloudFront, modules, state, or providers
- Do NOT answer Helm, Kubernetes YAML, or application-level questions
- Do NOT invent resources or issues
- If Terraform code is not provided and required, say:
  "Cannot analyze without Terraform code."

Task:
- Identify concrete Terraform issues or misconfigurations
- Avoid generic best practices unless directly relevant

Question:
{question}
"""
    )


In [ ]:
def kubernetes_agent(question: str):
    return rag_query(
        f"""
You are a Kubernetes expert reviewing configuration files.

Rules:
- Answer ONLY if the question is Kubernetes-related
- Identify whether the issue is in Helm, raw YAML, HPA, Ingress, or Service
- If the question refers to a file or path, assume it exists and analyze it
- Point out concrete misconfigurations or template bugs
- Do NOT give generic advice
- If insufficient information, say exactly what is missing

Question:
{question}
"""
    )


In [ ]:
def cicd_agent(question: str):
    return rag_query(
        f"""
You are a CI/CD and GitOps expert.

Rules:
- Answer ONLY if the question is related to CI/CD, GitOps, ArgoCD, Helm releases, or GitHub Actions
- Do NOT answer Kubernetes manifest debugging unless it directly affects CI/CD or ArgoCD sync
- Do NOT speculate or give generic advice
- If the question is unrelated to CI/CD, say:
  "This question is not related to CI/CD or GitOps."
- If file contents are required and not provided, ask for them

Task:
- Identify real CI/CD or ArgoCD issues
- Be specific and concise

Question:
{question}
"""
    )


In [ ]:
def incident_agent(error_log: str):
    return rag_query(
        f"""
An error occurred in production.

Error:
{error_log}

Analyze root cause using this repo.
Suggest safe, step-by-step fix.
Do NOT suggest destructive actions.
"""
    )


In [ ]:
def run_command(cmd: str):
    SAFE_COMMANDS = ["terraform plan", "kubectl get", "kubectl describe", "aws describe"]
    if not any(cmd.startswith(s) for s in SAFE_COMMANDS):
        return "❌ Command blocked for safety"

    try:
        output = subprocess.check_output(
            cmd, shell=True, stderr=subprocess.STDOUT, text=True
        )
        return output
    except subprocess.CalledProcessError as e:
        return e.output


In [ ]:
def devops_brain(question: str):
    return {
        "terraform": terraform_agent(question),
        "kubernetes": kubernetes_agent(question),
        "cicd": cicd_agent(question),
    }


In [ ]:
devops_brain(
    "what is my helm directory doing"
)


In [ ]:
incident_agent(
    "kubectl cannot connect to server: the server has asked for the client to provide credentials"
)
